<img src="https://dlnlp.ai/img/InterPARES_Audio.jpg" alt="InterPARES_Audio.jpg" width="35%" height="25%" align="right"/>


# 🚀 Multilingual Audio Analysis with InterPARES-Audio


### **Description**

This notebook provides an end-to-end pipeline to process a long audio file containing multiple speakers who may be speaking in different languages. It automates the following workflow:

1.  **Speaker Diarization**: Identifies *who* spoke and *when*, using `pyannote.audio`.
2.  **Multilingual Transcription**: Transcribes each speaker's segment, automatically detecting the language using `openai/whisper-large-v3`.
3.  **LLM Analysis**: Uses a Large Language Model (`openai/gpt-oss-20b`) running on a remote, Ollama-compatible service to summarize the transcript, extract action items, and generate key insights.

### **🛠️ Step 1 - Setup & Installation**

This cell installs all necessary Python libraries and configures the environment to use a GPU if one is available. A GPU is highly recommended for the performance-intensive models used in this pipeline.

In [1]:
# !pip3 install torch torchvision torchaudio
# !pip install pyannote.audio langchain ollama
# !pip install protobuf==3.20.3
# !pip install "numpy<2.0"
# !pip datasets
# !pip install --upgrade ipywidgets
# !pip install iso639
# !pip install pdfkit
# !pip install markdown2 weasyprint

### **📚 Step 2: Import Libraries and Load Models**

In [2]:
import ollama
import os
from datetime import datetime
from pyannote.audio import Pipeline as PyannotePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline
from datasets import Audio
from tqdm.auto import tqdm
# --- Suppress the Tqdm warning about ipywidgets ---
import warnings
from tqdm import TqdmWarning
warnings.filterwarnings("ignore", category=TqdmWarning)
import torchaudio
import torchaudio.transforms as T
from transformers import pipeline, WhisperForConditionalGeneration, WhisperProcessor
import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
# Import iso639 for language name lookup
from iso639 import Lang

In [ ]:
# Check GPUs usage
#----------------------
# Check if a GPU is available and set the device and data type accordingly
if torch.cuda.is_available():
    print("✅ GPU is available. We will use the CUDA device.")
    DEVICE = "cuda:0"
    TORCH_DTYPE = torch.float16
else:
    print("⚠️ GPU not available. Using CPU. This will be very slow.")
    DEVICE = "cpu"
    TORCH_DTYPE = torch.float32

In [ ]:
# --- Hugging Face Configuration ---
!huggingface-cli login --token <<<<YOUR_TOKEN>>>>
# --- Cache Dir  ---
# AUDIO_FILE ="A02898.wav"
CACHE_DIR="path_to_your_cache_dir"  # e.g. /content/cache

In [ ]:
# --- Part 1: Speaker Diarization (Language Agnostic) ---
print("✅ Loading Speaker Diarization Model")
diarization_pipeline = PyannotePipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    cache_dir=CACHE_DIR
)
diarization_pipeline.to(torch.device(DEVICE))

print("✅ Loading Whisper Model and Processor")

# --- Load model and processor ---
MODEL_NAME = "openai/whisper-large-v3"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TORCH_DTYPE = torch.float16
asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=MODEL_NAME,
    torch_dtype=TORCH_DTYPE,
    device=DEVICE,
)
processor = WhisperProcessor.from_pretrained(MODEL_NAME)


### **🦙 Step 3 - AI Service Connection**

This notebook is configured to connect to a remote, Ollama-compatible AI service. All configuration, including the service URL and model name, is handled in the next cell. No local Ollama setup is required.

In [ ]:
# --- AI Service Configuration ---
OLLAMA_SERVER = "OLLAMA_SERVER"  # e.g. "http://localhost:11434"
LLM_MODEL_NAME = "gpt-oss:20b"

In [ ]:
# --- Setup Ollama Client for external service ---
client = ollama.Client(host=OLLAMA_SERVER)
print(f"Connecting to AI service at {OLLAMA_SERVER} with model {LLM_MODEL_NAME}")

### 🎤 **Step 4: Functions**

### Utterance Generation and Transcription Functions

In [8]:
def generate_utterances(audio_file):
    diarization_output = diarization_pipeline(audio_file)
    print("✅ Diarization complete.")
    return diarization_output

def transcribe(audio_file):
    waveform, sr = torchaudio.load(audio_file)
    TARGET_SR = 16000
    if sr != TARGET_SR:
        waveform = T.Resample(sr, TARGET_SR)(waveform)
    waveform = waveform.squeeze()
    
    full_transcript = []
    print("Transcribing segments for each speaker...")
    for turn, _, speaker in tqdm(diarization_output.speaker_diarization.itertracks(yield_label=True)):
        start_time, end_time = turn.start, turn.end
        segment = waveform[int(start_time * TARGET_SR):int(end_time * TARGET_SR)].numpy()
        
        # Step 1: Detect language using Whisper's encoder
        input_features = processor(segment, return_tensors="pt", sampling_rate=TARGET_SR).input_features.to(DEVICE, dtype=TORCH_DTYPE)
        
        with torch.no_grad():
            # Get language logits from the model
            predicted_ids = asr_pipeline.model.detect_language(input_features)
            
            # Extract the language token ID
            if isinstance(predicted_ids, tuple):
                lang_token_id = predicted_ids[0].item()
            else:
                lang_token_id = predicted_ids.item() if predicted_ids.ndim == 0 else predicted_ids[0].item()
        
        # Decode the language token to get language code
        lang_token = processor.tokenizer.decode([lang_token_id])
        # Remove special token markers like <|en|>
        lang = lang_token.strip("<|>")
        try:
            lang_obj = Lang(lang)
            lang_name = lang_obj.name
        except (KeyError, AttributeError):
            # Fallback if language code not found
            lang_name = lang
        
        # Step 2: Transcribe with detected language
        result = asr_pipeline(segment, generate_kwargs={"language": lang, "task": "transcribe",
                                                       "return_timestamps": True  # Add this line
                                                       })
        text = result["text"].strip()
        
        full_transcript.append({
            "start": start_time,
            "speaker": speaker,
            "lang_code": lang,
            "lang_name": lang_name,
            "text": text
        })
               
        # break
    
    # --- Format output ---
    final_text = "\n".join(
        f"[{u['start']:.2f}s] ({u['lang_code']} – {u['lang_name']}) {u['speaker']}: {u['text']}"
        for u in full_transcript
    )
    print("\n--- ✅ Generate full transcript with language detection ---")
    return final_text

### Report Generation Functions

In [9]:
def estimate_tokens(text):
    """Estimate token count (rough approximation: 1 token ≈ 4 characters)"""
    return len(text) // 4


def generate_meeting_report(final_transcript_text, client, model_name="gpt-oss:20b", 
                           context_window=8192, reserved_tokens=1500, 
                           audio_filename=None, generated_by=None, target_language="English"):
    """
    Generate a comprehensive meeting analysis report from transcript.
    
    Args:
        full_transcript (list): List of dicts with keys: 'start', 'speaker', 'lang_name', 'text'
        client: Ollama client instance
        model_name (str): Name of the Ollama model to use
        context_window (int): Model's context window size in tokens
        reserved_tokens (int): Tokens to reserve for prompts and response
        audio_filename (str): Original audio file name (optional)
        generated_by (str): Name/identifier of person generating report (optional)
        target_language (str): Target language for the report (default: "English")
    
    Returns:
        tuple: (final_report (str), transcript_tokens (int), processing_method (str), metadata (dict))
    """
    print("\n--- Generating Meeting Analysis Report ---")
    
    # # Format the final transcript for LLM analysis
    # final_transcript_text = "\n".join(
    #     f"[{u['start']:.2f}s] {u['speaker']} ({u['lang_name']}): {u['text']}"
    #     for u in full_transcript
    # )
    
    # Calculate token limits
    max_input_tokens = context_window - reserved_tokens
    transcript_tokens = estimate_tokens(final_transcript_text)
    
    print(f"Transcript size: ~{transcript_tokens:,} tokens")
    print(f"Model context window: {context_window:,} tokens")
    print(f"Max input allowed: {max_input_tokens:,} tokens")
    
    # Decide processing method
    if transcript_tokens <= max_input_tokens:
        print("\n✓ Transcript fits in context window - using direct processing")
        final_report = _direct_processing(final_transcript_text, client, model_name, target_language)
        processing_method = "Direct"
    else:
        print(f"\n⚠ Transcript exceeds context window by ~{transcript_tokens - max_input_tokens:,} tokens")
        print("Using Map-Reduce approach...")
        final_report = _map_reduce_processing(final_transcript_text, client, model_name, 
                                              max_input_tokens, context_window, target_language)
        processing_method = "Map-Reduce"
    
    # Create metadata dictionary
    metadata = {
        'audio_filename': audio_filename,
        'generated_by': generated_by,
        'generation_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'model_name': model_name,
        'transcript_tokens': transcript_tokens,
        'processing_method': processing_method,
        'target_language': target_language
    }
    
    return final_report, transcript_tokens, processing_method, metadata


def _direct_processing(transcript_text, client, model_name, target_language):
    """Process transcript directly without chunking."""
    direct_prompt = f"""You are an expert audio meeting analyst (InterPARES-Audio). Analyze the following complete meeting transcript which may contain multiple languages. 

Based on the conversation content, infer likely names or roles for each speaker (e.g., "Sarah - Project Manager", "John - Developer", etc.).

Please provide a comprehensive report IN {target_language.upper()} with the following sections:

1. **Executive Summary**: Provide a concise, high-level summary of the meeting's purpose, key discussion points, and major outcomes. When mentioning participants, use their inferred names where identified (e.g., "SPEAKER_00, inferred as Maria").

2. **Speaker Profiles**: For each speaker, provide:
   - Speaker ID (e.g., SPEAKER_00)
   - Spoken Language(s)
   - Predicted Name/Role (inferred from context)
   - Individual Summary: What this person discussed, their main points, concerns, and contributions

3. **Main Topics Discussed**: Key subjects and themes covered in the meeting.

4. **Decisions Made**: Any conclusions or agreements reached.

5. **Action Items**: A consolidated, numbered list of all tasks, recommendations, and follow-ups (specify who is responsible if mentioned).

6. **Key Insights**: The top 3-5 most important insights or noteworthy observations.

Meeting Transcript:
---
{transcript_text}
---
IMPORTANT: Provide only the report content without any signatures, sign-offs, or closing remarks at the end.

Final Report (in {target_language}):"""

    print("\nGenerating comprehensive analysis...")
    response = client.chat(
        model=model_name,
        messages=[{'role': 'user', 'content': direct_prompt}]
    )
    
    return response['message']['content']


def _map_reduce_processing(transcript_text, client, model_name, max_input_tokens, context_window, target_language):
    """Process large transcript using map-reduce approach."""
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    
    # Calculate optimal chunk size
    chunk_size_chars = (max_input_tokens // 2) * 4
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size_chars,
        chunk_overlap=400,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    chunks = text_splitter.split_text(transcript_text)
    print(f"Split transcript into {len(chunks)} chunks")
    
    # MAP STAGE: Analyze each chunk
    map_prompt_template = """You are an expert audio meeting analyst (InterPARES-Audio). Analyze the following segment of a meeting transcript which may contain multiple languages. 

For each speaker in this segment, try to infer their likely name or role from context clues.

Extract:
- **Speakers Present**: List each speaker with their ID, language(s) spoken, and predicted name/role
- **Main Topics**: Key subjects discussed in this segment
- **Decisions**: Any conclusions or agreements
- **Action Items**: Specific tasks or recommendations (note who is responsible)
- **Key Points**: Important information or insights

Be concise but comprehensive.

Transcript Segment:
---
{chunk_text}
---

Analysis:"""
    
    print(f"\n--- MAP STAGE: Processing {len(chunks)} chunks ---")
    chunk_outputs = []
    for i, chunk in enumerate(chunks, 1):
        chunk_tokens = estimate_tokens(chunk)
        print(f"Chunk {i}/{len(chunks)} (~{chunk_tokens:,} tokens)...", end=" ")
        
        prompt = map_prompt_template.format(chunk_text=chunk)
        
        try:
            response = client.chat(
                model=model_name,
                messages=[{'role': 'user', 'content': prompt}]
            )
            chunk_outputs.append(f"=== Chunk {i} Analysis ===\n{response['message']['content']}")
            print("✓")
        except Exception as e:
            print(f"✗ Error: {e}")
            chunk_outputs.append(f"=== Chunk {i} Analysis ===\n[Error processing chunk]")
    
    # REDUCE STAGE: Synthesize all analyses
    combined_chunk_outputs = "\n\n".join(chunk_outputs)
    combined_tokens = estimate_tokens(combined_chunk_outputs)
    
    print(f"\n--- REDUCE STAGE: Synthesizing {combined_tokens:,} tokens ---")
    
    # Hierarchical reduction if needed
    if combined_tokens > max_input_tokens:
        print("⚠ Combined outputs too large, performing hierarchical reduction...")
        output_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size_chars,
            chunk_overlap=200
        )
        output_chunks = output_splitter.split_text(combined_chunk_outputs)
        
        intermediate_summaries = []
        for i, output_chunk in enumerate(output_chunks, 1):
            print(f"Intermediate summary {i}/{len(output_chunks)}...", end=" ")
            intermediate_prompt = f"""Synthesize the following meeting analyses. For each speaker, try to identify their name/role and summarize their contributions. Include topics, decisions, action items, and insights:

{output_chunk}

Synthesis:"""
            response = client.chat(
                model=model_name,
                messages=[{'role': 'user', 'content': intermediate_prompt}]
            )
            intermediate_summaries.append(response['message']['content'])
            print("✓")
        
        combined_chunk_outputs = "\n\n".join(intermediate_summaries)
    
    # Final reduction
    reduce_prompt_template = f"""You are a professional summarizer. You have been given analyses from a long, multilingual conversation. Synthesize this information into a comprehensive final report in {target_language}.

Based on all the information, infer likely names or roles for each speaker mentioned.

Provide the following sections IN {target_language.upper()}:

1. Executive Summary: Provide a concise, high-level summary of the meeting's purpose, key discussion points, and major outcomes. When mentioning participants, use their inferred names where identified (e.g., "SPEAKER_00, inferred as Maria").

2. **Speaker Profiles**: For each speaker identified across all segments, provide:
   Format: Speaker XX (Spoken Language: XX, Predicted Name: XXX): [Summary of what they discussed]
   Include their main points, concerns, and contributions to the conversation.

3. **Main Topics Discussed**: Key subjects and themes covered.

4. **Decisions Made**: Conclusions or agreements reached during the meeting.

5. **Action Items**: A consolidated, numbered list of all tasks and recommendations (specify who is responsible if mentioned).

6. **Key Insights**: The top 3-5 most important insights or noteworthy observations.

Information from Conversation Segments:
---
{combined_chunk_outputs}
---
IMPORTANT: Provide only the report content without any signatures, sign-offs, or closing remarks at the end.

Final Report (in {target_language}):"""
    
    print("Generating final report...")
    final_response = client.chat(
        model=model_name,
        messages=[{'role': 'user', 'content': reduce_prompt_template}]
    )
    
    return final_response['message']['content']


def save_report(final_report, metadata, output_dir="meeting_reports", base_filename=None, logo_path=None):
    """
    Save the meeting report to Markdown and PDF files.
    
    Args:
        final_report (str): The generated report content
        metadata (dict): Dictionary containing:
            - audio_filename (str): Original audio file name
            - generated_by (str): Person who generated the report
            - generation_date (str): Date/time of generation
            - model_name (str): Model used
            - transcript_tokens (int): Token count
            - processing_method (str): Processing method used
        output_dir (str): Directory to save reports
        base_filename (str): Base filename (default: timestamped)
        logo_path (str): Path to logo image file (optional, supports PNG, JPG, SVG)
    
    Returns:
        dict: Paths to saved files {'markdown': path, 'pdf': path or None}
    """
    print("\n--- Saving Report ---")
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Generate filename
    if base_filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_filename = f"meeting_analysis_{metadata['audio_filename'].split('/')[-1].split('.')[0]}-{timestamp}-{metadata['target_language']}"
    
    saved_files = {}
    
    # Save as Markdown
    md_file = os.path.join(output_dir, f"{base_filename}.md")
    with open(md_file, 'w', encoding='utf-8') as f:
        f.write("# UNESCO Meeting Analysis Report\n\n")
        f.write("## Metadata\n\n")
        if metadata.get('audio_filename'):
            f.write(f"**Audio File:** {metadata['audio_filename']}\n\n")
        f.write(f"**Date of Transcript:** {metadata['generation_date']}\n\n")
        f.write(f"**Target Language:** {metadata['target_language']}\n\n")
        # f.write(f"**Model:** {metadata['model_name']}\n\n")
        f.write(f"**Transcript Length:** ~{metadata['transcript_tokens']:,} tokens\n\n")
        # f.write(f"**Processing Method:** {metadata['processing_method']}\n\n")
        if metadata.get('generated_by'):
            f.write(f"**Generated By:** {metadata['generated_by']}\n\n")
        f.write("---\n\n")
        f.write(final_report)
    
    print(f"✓ Markdown report saved to: {md_file}")
    saved_files['markdown'] = md_file
    
    # Save as PDF
    pdf_file = os.path.join(output_dir, f"{base_filename}.pdf")
    pdf_saved = _save_as_pdf(final_report, pdf_file, metadata, logo_path)
    
    if pdf_saved:
        saved_files['pdf'] = pdf_file
    else:
        saved_files['pdf'] = None
    
    return saved_files

def _save_as_pdf(report_content, pdf_path, metadata, logo_path=None):
    """Save report as PDF with styling."""
    
    # --- FIX: Sanitize text to prevent character encoding errors in PDF ---
    if "Arabic" not in metadata['target_language']:
        report_content = report_content.encode('ascii', 'ignore').decode('utf-8')
    # --------------------------------------------------------------------

    try:
        # Try weasyprint first (recommended)
        try:
            import markdown2
            from weasyprint import HTML as WeasyHTML
            import base64
            
            # Prepare logo data if provided
            logo_data_uri = None
            if logo_path and os.path.exists(logo_path):
                with open(logo_path, 'rb') as img_file:
                    img_data = base64.b64encode(img_file.read()).decode('utf-8')
                    # Detect image type
                    ext = os.path.splitext(logo_path)[1].lower()
                    mime_types = {
                        '.png': 'image/png',
                        '.jpg': 'image/jpeg',
                        '.jpeg': 'image/jpeg',
                        '.svg': 'image/svg+xml',
                        '.gif': 'image/gif'
                    }
                    mime_type = mime_types.get(ext, 'image/png')
                    logo_data_uri = f"data:{mime_type};base64,{img_data}"
            
            html_content = markdown2.markdown(report_content, extras=['tables', 'fenced-code-blocks'])
            styled_html = _create_styled_html(html_content, metadata, logo_data_uri)
            
            WeasyHTML(string=styled_html).write_pdf(pdf_path)
            print(f"✓ PDF report saved to: {pdf_path}")
            return True
            
        except ImportError:
            # Fallback to pdfkit
            try:
                import markdown2
                import pdfkit
                import base64
                
                # Prepare logo data if provided
                logo_data_uri = None
                if logo_path and os.path.exists(logo_path):
                    with open(logo_path, 'rb') as img_file:
                        img_data = base64.b64encode(img_file.read()).decode('utf-8')
                        ext = os.path.splitext(logo_path)[1].lower()
                        mime_types = {
                            '.png': 'image/png',
                            '.jpg': 'image/jpeg',
                            '.jpeg': 'image/jpeg',
                            '.svg': 'image/svg+xml',
                            '.gif': 'image/gif'
                        }
                        mime_type = mime_types.get(ext, 'image/png')
                        logo_data_uri = f"data:{mime_type};base64,{img_data}"
                
                html_content = markdown2.markdown(report_content, extras=['tables', 'fenced-code-blocks'])
                styled_html = _create_styled_html(html_content, metadata, logo_data_uri)
                
                pdfkit.from_string(styled_html, pdf_path)
                print(f"✓ PDF report saved to: {pdf_path}")
                return True
                
            except ImportError:
                print("⚠ PDF generation skipped: Install 'weasyprint' or 'pdfkit + wkhtmltopdf'")
                print("  pip install weasyprint")
                print("  OR")
                print("  pip install pdfkit  # also requires wkhtmltopdf binary")
                return False
                
    except Exception as e:
        print(f"⚠ PDF generation failed: {e}")
        return False



def _create_styled_html(content, metadata, logo_data_uri=None):
    """Create styled HTML for PDF generation."""
    # Create logo HTML if provided
    logo_html = ""
    if logo_data_uri:
        logo_html = f'<img src="{logo_data_uri}" alt="Logo" class="logo" >'
    
    metadata_html = ""
    if metadata.get('audio_filename'):
        metadata_html += f"<strong>Audio File:</strong> {metadata['audio_filename']}<br>\n            "
    metadata_html += f"<strong>Date of Transcript:</strong> {metadata['generation_date']}<br>\n            "
    metadata_html += f"<strong>Target Language:</strong> {metadata['target_language']}<br>\n            "
    # metadata_html += f"<strong>Model:</strong> {metadata['model_name']}<br>\n            "
    metadata_html += f"<strong>Transcript Length:</strong> ~{metadata['transcript_tokens']:,} tokens            "
    # metadata_html += f"<strong>Processing Method:</strong> {metadata['processing_method']}"
    if metadata.get('generated_by'):
        metadata_html += f"<br>\n            <strong>Generated By:</strong> {metadata['generated_by']}"
    
    # Determine text direction based on target language
    target_lang = metadata.get('target_language', 'English').lower()
    rtl_languages = ['arabic', 'hebrew', 'persian', 'urdu', 'farsi', 'pashto']
    is_rtl = any(lang in target_lang for lang in rtl_languages)
    text_direction = 'rtl' if is_rtl else 'ltr'
    text_align = 'right' if is_rtl else 'left'
    # print (logo_html)
    return f"""
    <!DOCTYPE html>
    <html dir="{text_direction}">
    <head>
        <meta charset="utf-8">
        <style>
            @page {{
                size: A4;
                margin: 1.5cm;
            }}
            body {{
                font-family: 'Segoe UI', 'Tahoma', 'Arial', 'Noto Sans Arabic', 'Traditional Arabic', 'Simplified Arabic', sans-serif;
                line-height: 1.4;
                color: #333;
                font-size: 10pt;
                margin: 0;
                padding: 0;
                direction: {text_direction};
                text-align: {text_align};
            }}
    
            .header-container {{
                border-bottom: 2px solid #1c3b7c;
                padding-bottom: 10px;
                margin-bottom: 20px;
                display: flex;
                flex-direction: column;
                align-items: flex-start;
            }}
    
            .logo {{
                max-height: 70px;
                max-width: 220px;
                margin-bottom: 8px;
                
            }}
    
            .title {{
                width: 100%;
                text-align: center;
                font-size: 26px;
                font-weight: 700;
                color: #1c3b7c;
            }}
    
            h2 {{
                color: #607e2c;
                margin-top: 16px;
                margin-bottom: 8px;
                border-bottom: 1px solid #607e2c;
                padding-bottom: 4px;
                font-size: 13pt;
                text-align: {text_align};
            }}
    
            h3 {{
                color: #7f8c8d;
                margin-top: 12px;
                margin-bottom: 6px;
                font-size: 11pt;
                text-align: {text_align};
            }}
    
            ul, ol {{
                margin-{text_align}: 18px;
                margin-top: 6px;
                margin-bottom: 6px;
            }}
    
            li {{
                margin-bottom: 4px;
                line-height: 1.3;
                text-align: {text_align};
            }}
    
            p {{
                margin-top: 4px;
                margin-bottom: 6px;
                text-align: {text_align};
            }}
    
            strong {{
                color: #2c3e50;
            }}
    
            .metadata {{
                background-color: #ecf0f1;
                padding: 10px 12px;
                border-radius: 4px;
                margin-bottom: 16px;
                font-size: 9pt;
                line-height: 1.3;
                text-align: left;
                direction: ltr;
            }}
    
            code {{
                background-color: #f7f7f7;
                padding: 1px 4px;
                border-radius: 2px;
                font-family: 'Courier New', monospace;
                font-size: 9pt;
            }}
    
            table {{
                border-collapse: collapse;
                width: 100%;
                margin: 8px 0;
                font-size: 9pt;
                direction: {text_direction};
            }}
    
            th, td {{
                border: 1px solid #ddd;
                padding: 6px 8px;
                text-align: {text_align};
            }}
    
            th {{
                background-color: #f2f2f2;
                font-weight: bold;
            }}
        </style>
    </head>
    <body>
    
        <div class="header-container">
            <div class="logo">{logo_html}</div>
            <div class="title">📋 UNESCO Meeting Analysis Report</div>
        </div>
    
        <div class="metadata">
            {metadata_html}
        </div>
    
        {content}
    
    </body>
    </html>
    """




### ✅ Step 5: Process Audio and Finalize Analysis

In [ ]:
# Model configuration
MODEL_NAME = "gpt-oss:20b"
CONTEXT_WINDOW = 8192  # Confirmed from Ollama logs
RESERVED_TOKENS = 1500
GENERATED_BY = "InterPARES-Audio"  # Replace with your name or identifier

# Metadata configuration
for fname in [
                #"A02898", 
                "A02997", "A03046", "A03520", "A03858", "A04616", "A04957", "A04960", 
                        "A04965", "A05012", "A05195", "A05857", "A06010", "A06118", "A06622", "A06850",
                        "A06888", "A06918", "A06936", "A07167", "A07251", "A07296", "A07452", "A07462", 
                        "A07679", "A07768", "A07791", "A08133", "A08649", "A08957", "A08958", "A08960", 
                        "A08966", "A09304"]:
    AUDIO_FILENAME = f"resampled_audio/{fname}/{fname}.wav"
    OUTPUT_DIR=f"meeting_reports/{AUDIO_FILENAME.split('/')[-1][:-4]}"
    # Step 1: Generate the utterances
    diarization_output = generate_utterances(AUDIO_FILENAME)
    # Step e: Transcribe the utterances the report
    final_transcript_text = transcribe(AUDIO_FILENAME)
    # Step 3: Generate the report in multiple languages
    for TARGET_LANGUAGE in ["Arabic", "English", "French", "Spanish", "German", "Italian"]:
        print(f"***** {TARGET_LANGUAGE} *****")
        final_report, transcript_tokens, processing_method, metadata = generate_meeting_report(
            final_transcript_text=final_transcript_text,
            client=client,
            model_name=MODEL_NAME,
            context_window=CONTEXT_WINDOW,
            reserved_tokens=RESERVED_TOKENS,
            audio_filename=AUDIO_FILENAME,
            generated_by=GENERATED_BY,
            target_language=TARGET_LANGUAGE
        )
        # Step 4: Save the report
        saved_files = save_report(
            final_report=final_report,
            metadata=metadata,
            output_dir=f"meeting_reports/{AUDIO_FILENAME.split('/')[-1][:-4]}",
            logo_path="InterPARES_Audio.jpg"  # Path to your logo file
        )
        
        print(f"\n{'='*70}")
        print("Report generation complete!")
        print(f"Markdown: {saved_files['markdown']}")
        if saved_files['pdf']:
            print(f"PDF: {saved_files['pdf']}")
        with open(f"{OUTPUT_DIR}/final_transcript_text.txt", "a") as tf:
            print(final_transcript_text, file=tf)
        print(f"{'='*70}")